In [44]:
import pandas as pd

def load_data():
    return pd.read_csv("datasets/ckd.csv")
data = load_data()

In [45]:
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(data, test_size=0.2, random_state=42)
data = train_set.drop(labels=["CKD_Stage", "CKD_Progression"], axis=1)
labels = train_set.drop(labels=data.columns, axis=1)

In [ ]:
#scales the data and fills in missing values with the median
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
num_pipeline = Pipeline([
('imputer', SimpleImputer(strategy="median")),
('std_scaler', MinMaxScaler()),
])
data = num_pipeline.fit_transform(data)

AttributeError: 'DataFrame' object has no attribute 'dtype'